# Calculating EOFs & PCs using the 'eofs' package

## Import the sst data from a netcdf file or multiple netcdf files

In [ ]:
#! conda install -c conda-forge eofs

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt
from eofs.standard import Eof
%matplotlib inline
from glob import glob
import os
#import warnings
#warnings.simplefilter("ignore") 

## load the SST data from files using xarray

In [ ]:
ds = xr.open_dataset('https://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/\
                       .version4/.anom/T/-23.5/last/RANGE/dods',decode_times=False)

ds['T'] = pd.date_range('1/1/1958', periods=ds.anom.shape[0], freq='MS').shift(15, freq='D')
ds

In [ ]:
ds2 = ds.drop(['zlev']).squeeze().sel(T=slice('1958-01-01','2014-12-01'))

In [ ]:
dpac = ds2.sel(Y=slice(-40,40), X=slice(120,290))
ds_annual = dpac.resample(T='AS').mean(dim='T')
ds_anom = ds_annual - ds_annual.mean(dim='T')

### define weights

In [ ]:
coslat = np.cos(np.deg2rad(dpac.Y))
ds_anom['wgts'] = np.sqrt(coslat) + 0*dpac.X
np.shape(ds_anom.anom.values)

### instantiates the EOF solver, passing the weights array as an optional argument

In [ ]:
solver = Eof(ds_anom.anom.values, weights=ds_anom.wgts.values)

### get the EOFs (spatial patterns) as correlations

In [ ]:
number_of_eofs = 4
Ss = solver.eofsAsCovariance(neofs=number_of_eofs)
ds_anom['ev'] = np.arange(0,number_of_eofs)
ds_anom['Ss'] = (['ev','Y','X'],Ss)

### get the Principal Components (PCs), normalized values

In [ ]:
Ts = solver.pcs(npcs=number_of_eofs, pcscaling=1)
ds_anom['Ts'] = (['ev','T'],Ts.T)

### plots 

In [ ]:
plt.figure(figsize=(8, 10))
plt.subplot(211)
ds_anom.Ss[0].plot()
plt.subplot(212)
ds_anom.Ss[1].plot()

In [ ]:
ds_anom.Ts[0].plot(figsize=(10,5)); plt.title('PC1',fontsize=16)
ds_anom.Ts[1].plot(figsize=(10,5)); plt.title('PC2',fontsize=16)

In [ ]:
ds_anom.to_netcdf('python_EOF.nc',encoding={'T':{'dtype':'float32'}})